In [1]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---
import joblib
import numpy as np
import os 
import pandas as pd

# Imports Essenciais para Stacking
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import StackingClassifier # <-- O ENSEMBLE AVANÇADO
from sklearn.linear_model import LogisticRegression # <-- Modelo Base e Meta-Modelo

# Outros Modelos (apenas para tipagem, se necessário, ou se o Stacking exigir)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

print("Bibliotecas importadas, prontas para Stacking Ensemble.")

Bibliotecas importadas, prontas para Stacking Ensemble.


In [4]:
# --- CÉLULA 2: Carregamento de Dados e Modelos Salvos ---

MODEL_DIR = 'models'
DATA_DIR = 'data'

# 1. Carregamento dos Conjuntos de Dados (Scaled)
try:
    X_train_scaled = joblib.load(os.path.join(DATA_DIR, 'X_train_pima_scaled.joblib'))
    X_test_scaled = joblib.load(os.path.join(DATA_DIR, 'X_test_pima_scaled.joblib'))
    y_train = joblib.load(os.path.join(DATA_DIR, 'y_train_pima.joblib'))
    y_test = joblib.load(os.path.join(DATA_DIR, 'y_test_pima.joblib'))
    print("Dados Pima Indians Diabetes (scaled) carregados com sucesso.")
except Exception as e:
    print(f"ERRO: Falha ao carregar dados. {e}")
    
# 2. Carregamento dos Modelos de Base (Base Learners)
try:
    # A - XGBoost (Melhor AUC-ROC)
    xgb_clf = joblib.load(os.path.join(MODEL_DIR, 'xgboost_classifier_pima.joblib'))
    
    # B - Naive Bayes (Melhor Recall)
    nb_clf = joblib.load(os.path.join(MODEL_DIR, 'naive_bayes_classifier_model.joblib'))
    
    # C - KNN (Melhor Acurácia)
    knn_clf = joblib.load(os.path.join(MODEL_DIR, 'knn_classifier_pima.joblib'))
    
    # D - Regressão Logística (Alto AUC-ROC e Linearidade)
    logreg_clf = joblib.load(os.path.join(MODEL_DIR, 'logistic_regression_classifier_model.joblib'))
    
    print("Modelos base (XGBoost, NB, KNN, LogReg) carregados com sucesso.")

except Exception as e:
    print(f"ERRO: Falha ao carregar modelos. Certifique-se de que todos os arquivos JOBlib existem. {e}")
    raise

Dados Pima Indians Diabetes (scaled) carregados com sucesso.
Modelos base (XGBoost, NB, KNN, LogReg) carregados com sucesso.


In [5]:
# --- CÉLULA 3: Implementação e Treinamento do Stacking Classifier ---

# 1. Definir os Modelos de Nível 0 (Base Learners)
# É crucial fornecer um nome e o objeto do modelo carregado (como você ajustou na Célula 2)
base_learners = [
    ('logreg', logreg_clf),
    ('knn', knn_clf),
    ('naive_bayes', nb_clf),
    ('xgboost', xgb_clf)
]

# 2. Definir o Meta-Modelo (Modelo de Nível 1 - Final Estimator)
# Usaremos uma Regressão Logística simples para aprender a melhor forma de combinar as previsões
final_estimator = LogisticRegression(solver='liblinear', random_state=42)

# 3. Criar o Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=final_estimator,
    cv=5, # Usará 5-fold cross-validation
    stack_method='predict_proba', # Passa as PROBABILIDADES como features para o meta-modelo
    n_jobs=-1,
    passthrough=False 
)

# 4. Treinar o Stacking Ensemble
print("Iniciando o treinamento do Stacking Classifier...")
stacking_clf.fit(X_train_scaled, y_train)
print("Treinamento do Stacking Classifier concluído.")

# 5. Salvar o Stacking Ensemble
MODEL_DIR = 'models'
MODEL_PATH_STACK = os.path.join(MODEL_DIR, 'stacking_ensemble_pima.joblib')
joblib.dump(stacking_clf, MODEL_PATH_STACK)
print(f"Modelo Stacking Ensemble salvo em: {MODEL_PATH_STACK}")

Iniciando o treinamento do Stacking Classifier...
Treinamento do Stacking Classifier concluído.
Modelo Stacking Ensemble salvo em: models\stacking_ensemble_pima.joblib


In [6]:
# --- CÉLULA 4: Avaliação e Comparação Final ---

# 1. Avaliação no Conjunto de Teste
y_pred_proba_stack = stacking_clf.predict_proba(X_test_scaled)[:, 1]
y_pred_stack = stacking_clf.predict(X_test_scaled)

auc_stack = roc_auc_score(y_test, y_pred_proba_stack)

# 2. Comparação com o Melhor Modelo Individual (XGBoost)
y_pred_proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]
auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)


print("\n--- Comparação Final de Desempenho (AUC-ROC) ---")
print(f"XGBoost (Melhor Individual): AUC-ROC = {auc_xgb:.4f}")
print(f"Stacking Ensemble: AUC-ROC = {auc_stack:.4f}")

# 3. Relatório Detalhado do Stacking
print("\nRelatório de Classificação (Stacking Ensemble):")
print(classification_report(y_test, y_pred_stack))

if auc_stack > auc_xgb:
    print(f"\n✅ SUCESSO! O Stacking Ensemble ({auc_stack:.4f}) superou o melhor modelo individual (XGBoost)!")
else:
    print(f"\n⚠️ O Stacking Ensemble não superou o XGBoost, mas fornece uma alternativa robusta e combina diferentes visões.")


--- Comparação Final de Desempenho (AUC-ROC) ---
XGBoost (Melhor Individual): AUC-ROC = 0.8416
Stacking Ensemble: AUC-ROC = 0.8421

Relatório de Classificação (Stacking Ensemble):
              precision    recall  f1-score   support

           0       0.77      0.86      0.81       150
           1       0.67      0.52      0.58        81

    accuracy                           0.74       231
   macro avg       0.72      0.69      0.70       231
weighted avg       0.73      0.74      0.73       231


✅ SUCESSO! O Stacking Ensemble (0.8421) superou o melhor modelo individual (XGBoost)!
